## Import required libraries

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_recall_fscore_support
from sklearn.cross_validation import train_test_split

## Load train and test data

In [2]:
df_data = pd.read_csv("./enwiki.draft_quality.50k_stratified.feature_labels.tsv", sep="\t")
df_data["OK"] = df_data['draft_quality'].apply(lambda x: x == "OK")
df_data["spam"] = df_data['draft_quality'].apply(lambda x: x == "spam")
df_data["vandalism"] = df_data['draft_quality'].apply(lambda x: x == "vandalism")
df_data["attack"] = df_data['draft_quality'].apply(lambda x: x == "attack")

In [3]:
features = df_data.columns[:-1]
len(features)
data = df_data[features]

## Select feature columns

## Select target columns

In [4]:
targets = ["spam"]#, "OK", "vandalism", "attack"]

## Train and Run models 

### Defining models 

In [5]:
models = [dict()]#, dict(), dict(), dict()]
models[0]["model"] = RandomForestClassifier()
models[0]["name"] = "RandomForestClassifier"
#models[1]["model"] = GradientBoostingClassifier()
#models[1]["name"] = "GradientBoostingClassifier"
#models[2]["model"] = SVC()
#models[2]["name"] = "SVC"
#models[3]["model"] = GaussianNB()
#models[3]["name"] = "GaussianNB"

### Adding model parametes

In [6]:
for model in models : 
    model["params"] = model["model"].get_params()

### Training models

In [ ]:
for model in models : 
    for category in targets : 
        X_train, X_test, y_train, y_test = train_test_split(df_data[features], df_data[category], test_size=0.4, random_state=0)
        
        
        model["model"].fit(X_train, y_train)
        y_pred = model["model"].predict(X_test)
        model["metrics_for_" + category] = precision_recall_fscore_support(y_test, y_pred, average='binary')
    mean_precision = 0.0
    mean_recall = 0.0
    for category in targets : 
        mean_precision += model["metrics_for_" + category][0]
        mean_recall += model["metrics_for_" + category][1]
    model["mean_precision"] = mean_precision / len(targets)
    model["mean_recall"] = mean_recall / len(targets)

## Evaluation

In [ ]:
for model in models: 
    print "%s\n\tPrecision: %.2f\n\tRecall: %.2f\n\n" % (model["name"],  model["mean_precision"], model["mean_recall"])